A collection of utility functions for working with aeon raw data 

In [6]:
import aeon.io.api as aeon
import pandas as pd
from aeon.analysis.utils import *
from aeon.schema.dataset import exp02
from dotmap import DotMap

In [28]:
def get_experiment_times(
    root: str, visit_start: pd.Timestamp, visit_end: pd.Timestamp
) -> DotMap:
    """
    Retrieve experiment start and stop times (i.e. times outside of 
    maintenance mode) from environment states.

    Args:
        root (str): The root path where epoch data is stored.
        visit_start (pd.Timestamp): Subject visit start time.
        visit_end (pd.Timestamp): Subject visit end time.

    Returns:
        DotMap: A DotMap object containing two keys: 'start' and 'stop',
        corresponding to pairs of experiment start and stop times.
    
    Notes:
    This function uses the last 'Maintenance' event as the last 'Experiment'
    stop time. If the first retrieved state is 'Maintenance' (e.g. 
    'Experiment' mode entered before `visit_start`), `visit_start` is used 
    as the first 'Experiment' start time.
    """

    experiment_times = DotMap()
    env_states = aeon.load(
        root,
        exp02.ExperimentalMetadata.EnvironmentState,
        visit_start,
        visit_end,
    )
    # Use the last 'maintenance' event as visit_end time
    visit_end = (env_states[env_states.state == "Maintenance"]).index[-1]
    env_states = env_states[~env_states.index.duplicated(keep="first")]
    # Retain only events between visit start and stop times
    env_states = env_states.iloc[
        env_states.index.get_indexer([visit_start], method="bfill")[
            0
        ] : env_states.index.get_indexer([visit_end], method="ffill")[0] + 1
    ]
    # Retain only events where state changes (experiment-maintenance pairs)
    env_states = env_states[env_states["state"].ne(env_states["state"].shift())]
    if env_states["state"].iloc[0] == "Maintenance":
        # Pad with an "Experiment" event at the start
        env_states = pd.concat(
            [
                pd.DataFrame(
                    "Experiment",
                    index=[visit_start],
                    columns=env_states.columns,
                ),
                env_states,
            ]
        )
    else:
        # Use visit_start time as the first "Experiment" event
        env_states.rename(index={env_states.index[0]: visit_start}, inplace=True)
    experiment_times.start = env_states[env_states["state"] == "Experiment"].index.values
    experiment_times.stop = env_states[env_states["state"] == "Maintenance"].index.values

    return experiment_times


def exclude_maintenance_data(data: pd.DataFrame, experiment_times: DotMap) -> pd.DataFrame:
    """
    Exclude rows not in experiment times (i.e., corresponding to maintenance times)
    from the given dataframe.

    Args:
        data (pandas.DataFrame): The data to filter. Expected to have a DateTimeIndex.
        experiment_times (DotMap): A DotMap object containing experiment start and stop times.

    Returns:
        pandas.DataFrame: The filtered data.
    """
    filtered_data = pd.concat(
        [
            data.loc[start:stop]
            for start, stop in zip(experiment_times.start, experiment_times.stop)
        ]
    )
    return filtered_data

In [ ]:
# example
root = "/ceph/aeon/aeon/data/raw/AEON2/experiment0.2/"
subject_data = aeon.load(root, exp02.ExperimentalMetadata.SubjectState)
subject_data = subject_data[subject_data.id.str.startswith("BAA")]
subject_data = subject_data[subject_data.event != "Remain"]
subject_visits = visits(subject_data)
visit_start = subject_visits.enter[0]
visit_end = subject_visits.exit[0]
experiment_times = get_experiment_times(root, visit_start, visit_end)
pos_data = aeon.load(root, exp02.CameraTop.Position, visit_start, visit_end)
pos_data = exclude_maintenance_data(pos_data, experiment_times)